<a href="https://colab.research.google.com/github/HenriARM/ML/blob/master/ResNetGPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
%pip install torchnet

     |████████████████████████████████| 686kB 11.6MB/s 
     |████████████████████████████████| 204kB 39.1MB/s 
  Created wheel for torchnet: filename=torchnet-0.0.4-cp36-none-any.whl size=29744 sha256=b2b3c97dab98bbb57d633b3209fcb1f6b0c78b0099caee9c32d2537c70edd260
  Stored in directory: /root/.cache/pip/wheels/e1/03/fb/1c212c2f20905cdf97fe39022946cf16b8e66ed754a6663400
  Created wheel for visdom: filename=visdom-0.1.8.9-cp36-none-any.whl size=655252 sha256=51d813880fa3c18d61db808a5fbb1768b237655d55bf2f7a68c50ec944669511
  Stored in directory: /root/.cache/pip/wheels/70/19/a7/6d589ed967f4dfefd33bc166d081257bd4ed0cb618dccfd62a
  Created wheel for torchfile: filename=torchfile-0.1.0-cp36-none-any.whl size=5713 sha256=341b578e572573869c1478b28d86f31e7ea61b24ed921d4b0ddc09afda33956c
  Stored in directory: /root/.cache/pip/wheels/b1/c3/d6/9a1cc8f3a99a0fc1124cae20153f36af59a6e683daca0a0814
Successfully built torchnet visdom torchfile


In [5]:
import torchvision
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
import torch.nn as nn
from torch.nn.functional import pad, softmax
from torchsummary import summary

import torch
from torch.optim import Adam
from torchnet.meter import AverageValueMeter

In [46]:
DEVICE = 'cpu'
if torch.cuda.is_available():
    DEVICE = 'cuda'

EPOCHS = 10
BATCH_SIZE = 64
lr = 1e-4

In [47]:
DEVICE

'cuda'

In [48]:
def conv_3x3(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, padding=1, stride=1),
        nn.ReLU(),
        nn.BatchNorm2d(num_features=out_channels)
    )


class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ResidualBlock, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.layers = nn.Sequential(
            conv_3x3(in_channels=self.in_channels, out_channels=self.out_channels),
            nn.ReLU(),
            conv_3x3(in_channels=self.out_channels, out_channels=self.out_channels)
        )
        self.shortcut = conv_3x3(in_channels=self.in_channels, out_channels=self.out_channels)

    def forward(self, x):
        residual = self.shortcut(x)
        x = self.layers(x)
        return x + residual


class ResidualGate(nn.Module):
    def __init__(self, in_channels, out_channels, blocks):
        super(ResidualGate, self).__init__()
        self.blocks = nn.Sequential(
            ResidualBlock(in_channels=in_channels, out_channels=out_channels),
            *[ResidualBlock(in_channels=out_channels, out_channels=out_channels) for _ in range(blocks - 1)]
        )

    def forward(self, x):
        for block in self.blocks:
            x = block.forward(x)
        return x


class ResNet(nn.Module):
    def __init__(self, in_channels, n_classes):
        super(ResNet, self).__init__()
        self.input = nn.Sequential(
            # out = (28 + 2*1 - 3) / 1 + 1   (28)
            nn.Conv2d(in_channels=in_channels, out_channels=16, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            # out = (28 + 2*1 - 3) / 2 + 1   (14)
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        size = [16, 32, 64, 128]
        self.gates = nn.ModuleList(
            [ResidualGate(in_channels=i[0], out_channels=i[1], blocks=2) for i in tuple(zip(size, size[1:]))]
        )
        self.fc = nn.Linear(in_features=128, out_features=n_classes)

    def forward(self, x):
        x = self.input(x)
        for gate in self.gates:
            x = gate.forward(x)
        x = torch.nn.functional.adaptive_avg_pool2d(x, output_size=(1, 1))
        x = x.reshape(x.size(0), -1)
        x = self.fc(x)
        x = softmax(x, dim=1)
        return x


def main():
    # Use standard FashionMNIST dataset
    train_set = torchvision.datasets.FashionMNIST(
        root='./datasets',
        train=True,
        download=True,
        transform=transforms.Compose([transforms.ToTensor()])
    )

    test_set = torchvision.datasets.FashionMNIST(
        root='./datasets',
        train=False,
        download=True,
        transform=transforms.Compose([transforms.ToTensor()])
    )

    train_loader = DataLoader(dataset=train_set, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(dataset=test_set, batch_size=BATCH_SIZE, shuffle=True)

    model = ResNet(in_channels=1, n_classes=10)
    model.to(DEVICE)
    # summary(model, (1, 28, 28))

    optimizer = Adam(model.parameters(), lr=lr)
    meters: dict = {
        'train_loss': [],
        'test_loss': []
    }
    for epoch in range(EPOCHS):
        print("\nepoch = ", epoch)
        for loader in [train_loader, test_loader]:
            if loader == train_loader:
                print("\n\ttraining:")
                meter_prefix = "train"
                model = model.train()
                torch.set_grad_enabled(True)
            else:
                print("\n\ttesting:")
                meter_prefix = "test"
                model = model.eval()
                torch.set_grad_enabled(False)
            losses = AverageValueMeter()
            for x, y_idx in loader:
                # if losses.n > 10:
                #     break

                x = x.to(DEVICE)
                y_idx = y_idx.to(DEVICE)
                y_prim = model.forward(x)

                # use custom implemented cross-entropy      
                # loss = -torch.mean(torch.log(y_prim + 1e-8)[torch.arange(BATCH_SIZE), y_idx])
                # print(loss)

                # convert label to one-hot encoded
                y = torch.zeros((x.size(0), 10))
                y[torch.arange(x.size(0)), y_idx] = 1.0
                y = y.to(DEVICE)

                # batch loss
                loss = -torch.mean(y * torch.log(y_prim + 1e-8))

                # loss.to('cpu').item() => single scalar value
                # loss.to('cpu').data.numpy() => matrix
                losses.add(loss.to(DEVICE).item())

                if loader == train_loader:
                    loss.backward()
                    optimizer.step()
                    optimizer.zero_grad()

            # losses.value is average loss of all batches
            meters[f'{meter_prefix}_loss'].append(losses.value()[0])
            print(losses.value()[0])
    print(meters)

In [49]:
if __name__ == '__main__':
    print(DEVICE)
    with tf.device('/device:GPU:0'):
      main()

cuda

epoch =  0

	training:
0.04847013528571968

	testing:
0.033804590226548484

epoch =  1

	training:
0.02919201808832664

	testing:
0.029807117530352378

epoch =  2

	training:
0.02444483154390786

	testing:
0.026959021670660784

epoch =  3

	training:
0.020589374094657397

	testing:
0.025049498636916183

epoch =  4

	training:
0.01743116132867363

	testing:
0.028677609793628288

epoch =  5

	training:
0.01468802342858555

	testing:
0.02667188205441851

epoch =  6

	training:
0.012274360920397886

	testing:
0.028266032798820808

epoch =  7

	training:
0.009898733621937803

	testing:
0.028449030158519274

epoch =  8

	training:
0.008322510416612021

	testing:
0.02906098164570559

epoch =  9

	training:
0.006783474998607957

	testing:
0.03236117986902879
{'train_loss': [0.04847013528571968, 0.02919201808832664, 0.02444483154390786, 0.020589374094657397, 0.01743116132867363, 0.01468802342858555, 0.012274360920397886, 0.009898733621937803, 0.008322510416612021, 0.006783474998607957], '